In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader
from torchvision.datasets import Flowers102
from torchvision import transforms
import matplotlib.pyplot as plt


KeyboardInterrupt



In [ ]:
transform = transforms.Compose([transforms.ToTensor(),
                                transforms.Resize((224,224))])

In [ ]:
training_data = Flowers102(
    root='',
    split='train',
    transform=transform,
    download=True
)

testing_data = Flowers102(
    root='',
    split='test',
    transform=transform,
    download=True
)

evaluation_data = Flowers102(
    root='',
    split='val',
    transform=transform,
    download=True
)

In [ ]:
training_loader = torch.utils.data.DataLoader(training_data,batch_size=4,shuffle=True, num_workers=2)
testing_loader = torch.utils.data.DataLoader(testing_data,batch_size=4,shuffle=False, num_workers=2)

In [ ]:
print('Training data has {} instances'.format(len(training_data)))
print('Evaluation data has {} instances'.format(len(evaluation_data)))
print('Testing data has {} instances'.format(len(testing_data)))

In [ ]:
img, label = training_data[160]
np_img = img.permute(1, 2, 0).numpy()
plt.title(label)
plt.imshow((np_img * 255).astype('uint8'))
plt.axis('off')
plt.show()

In [ ]:
figure = plt.figure(figsize=(10, 10))
cols, rows = 3, 5
for i in range(1, cols * rows + 1):
    sample_idx = torch.randint(len(training_data), size=(1,)).item()
    img, label = training_data[sample_idx]
    
    np_img = img.permute(1, 2, 0).numpy()
    figure.add_subplot(rows, cols, i)
    plt.title(label)
    plt.imshow((np_img * 255).astype('uint8'))
    plt.axis('off')
plt.show()

In [ ]:
# data augmentation so more than 10 photos per class, diversify training data

In [ ]:
# transform simple NN into a CNN
class FlowerClassifier(nn.Module):
    def __init__(self):
        super(FlowerClassifier, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, kernel_size=3, stride=1, padding=1)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1)
        self.dropout1 = nn.Dropout(0.25)
        self.dropout2 = nn.Dropout(0.5)
        self.fc1 = nn.Linear(64 * 56 * 56, 128)
        self.fc2 = nn.Linear(128, 102)

    def forward(self, x):
        x = self.conv1(x)
        x = F.relu(x)
        x = F.max_pool2d(x, 2)
        x = self.conv2(x)
        x = F.relu(x)
        x = F.max_pool2d(x, 2)
        x = self.dropout1(x)
        x = torch.flatten(x, 1)
        x = self.fc1(x)
        x = F.relu(x)
        x = self.dropout2(x)
        x = self.fc2(x)
        output = F.log_softmax(x, dim=1)
        return output

In [ ]:
training_loader = DataLoader(training_data, batch_size=32, shuffle=True)
evaluation_loader = DataLoader(evaluation_data, batch_size=32, shuffle=False)

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
def train(model, training_loader, evaluation_loader, criterion, optimizer, epochs=10):
    for epoch in range(epochs):
        model.train()
        training_loss = 0.0
        for inputs,labels in training_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            training_loss += loss.item() * inputs.size(0)
        
        training_loss /= len(training_loader.dataset)
        
        model.eval()
        evaluation_loss = 0.0
        correct = 0
        total = 0
        with torch.no_grad():
            for inputs,labels in evaluation_loader:
                inputs, labels = inputs.to(device), labels.to(device)
                outputs = model(inputs)
                loss = criterion(outputs, labels)
                evaluation_loss += loss.item() * inputs.size(0)
                _, predicted = torch.max(outputs,1)
                total += labels.size(0)
                correct += (predicted == labels).sum().item()
        
        evaluation_loss /= len(evaluation_loader.dataset)
        evaluation_accuracy = correct / total
        
        print(f'Epoch {epoch+1}/{epochs}, Training Loss: {training_loss:.4f}, Evaluation Loss: {evaluation_loss:.4f}, Val Accuracy: {evaluation_accuracy:.4f}')

In [ ]:
model = FlowerClassifier()
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

train(model, training_loader, evaluation_loader, criterion, optimizer, epochs=10)